In [0]:
!pip install Pillow==4.1.1
!pip install "fastai==0.7.0"
!pip install torchtext==0.2.3
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
!apt update && apt install -y libsm6 libxext6

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
!pip install image

%matplotlib inline

In [0]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [0]:
!wget http://files.fast.ai/data/dogscats.zip

--2019-01-02 17:21:30--  http://files.fast.ai/data/dogscats.zip
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857214334 (818M) [application/zip]
Saving to: ‘dogscats.zip’

dogscats.zip        100%[===================>] 817.50M  35.9MB/s    in 23s     

2019-01-02 17:21:54 (35.0 MB/s) - ‘dogscats.zip’ saved [857214334/857214334]



In [0]:
!unzip dogscats.zip -d data

In [0]:
PATH = "data/dogscats/"
sz = 224

In [0]:
!ls {PATH}valid

In [0]:
files = !ls {PATH}valid/cats | head
files

In [0]:
img = plt.imread(f'{PATH}valid/cats/{files[0]}')
plt.imshow(img);

In [0]:
arch=resnet34

In [0]:

data = ImageClassifierData.from_paths(PATH, tfms = tfms_from_model(resnet34,sz)) # contains train & val data
learn = ConvLearner.pretrained(resnet34,data,precompute = True) # model == CNN
learn.fit(0.01,3)


# output will be 3 epochs with 1st and 2nd no is value of loss fxn of training and validn set
# last number is accuracy

In [0]:
data.val_y  # dependent var validn set
??ImageClassifierData

In [0]:
 data.classes

In [0]:
log_preds = learn.predict()
log_preds.shape

In [0]:
  log_preds[:10] # these in logarithms

In [0]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])  # pred in log so doing exp

In [0]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], min(len(preds), 4), replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

In [0]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [0]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8)) if len(imgs)>0 else print('Not Found.')

In [0]:
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [0]:
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")


In [0]:

def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

In [0]:
plot_val_with_title(most_by_correct(0, True), "Most correct cats")


In [0]:
plot_val_with_title(most_by_correct(1, True), "Most correct dogs")


In [0]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect cats")


In [0]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect dogs")


In [0]:
most_uncertain = np.argsort(np.abs(probs -0.5))[:4]
plot_val_with_title(most_uncertain, "Most uncertain predictions")

In [0]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [0]:
lrf=learn.lr_find()

In [0]:
learn.sched.plot_lr()

In [0]:
learn.sched.plot()

In [0]:
#using learning rate = 10^-2
# overfitting when training loss= lower than val loss (lower the loss thr better)

learn.fit(0.01,3)

In [0]:
# transforming images
tfms = tfms_from_model(resnet34,sz,aug_tfms = transforms_side_on,max_zoom = 1.1) # adjust/transform data suitable for
# resent arch

In [0]:
def get_augs():
    data = ImageClassifierData.from_paths(PATH, bs=2, tfms=tfms, num_workers=1)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [0]:
ims = np.stack([get_augs() for i in range(6)])


In [0]:
plots(ims, rows=2)


In [0]:
data = ImageClassifierData.from_paths(PATH,tfms = tfms)  
# alternate version if not have train and val and test folders 
# data = ImageClassifierData.from_paths(trn_name = "",val_name =,test_name =,tfms = tfms)
learn = ConvLearner.pretrained(arch,data,precompute = True) # precompute = t caches intermediate 
# steps to not be recalculated again
learn.fit(1e-2,1)

# if using bigger arch eg resnet 50 and above use 
# learn.unfreeze() as well as learn.bn_freeze [it will cause batch normalization moving averages to be not updated]


In [0]:
learn.precompute = False
# By default when we create a learner, it sets all but the last layer to frozen. 
# That means that it's still only updating the weights in the last layer when we call fit

In [0]:
learn.fit(1e-2,3,cycle_len=1)  # cycle len = 1 === stochastic grad desc with restarts; is a variant of annealing 
# here we may get to a point where small change in weights leads to large changes in loss so we restart our learning
# rates after some epochs number of epochs b/w reset == cycle_len no. of times this happens = num of cycles

In [0]:
learn.sched.plot_lr()

In [0]:
# now we train another layers but first unfreeze them
learn.unfreeze()

In [0]:
'''
Note that the other layers have already been trained to recognize imagenet photos (whereas our final layers where randomly initialized), 
so we want to be careful of not destroying the carefully tuned weights that are already there.

we use differential learning rates for input middle and end output layers

the first few layers will be at 1e-4, the middle layers at 1e-3, and our FC layers we'll leave at 1e-2 as before.
'''
lr = np.array([1e-4,1e-3,1e-2])

In [0]:
learn.fit(lr,3,cycle_len =1,cycle_mult =2)

In [0]:
learn.save('224_all')


In [0]:
log_preds,y = learn.TTA()  # test time augumentation will add additional augumentations (4) for training
probs = np.mean(np.exp(log_preds),0) # take avg of those and use them too

In [0]:
accuracy_np(probs, y)
